# Météo Toulouse

In [2]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client import InfluxDBClient, Point, WriteOptions
import rx
from rx import operators as ops
from collections import OrderedDict
from csv import DictReader
from datetime import datetime

In [3]:
bucket = "meteo_toulouse"
org = "obd_influxdb"
token = "DkqpJF-KB6oCD25k7E8woeHxK0deggIpNyDxFeoszhLogSRrYLKdYMdcZjVGXyutwFSFBfLD5-Jkbah6wt9ujA=="
url="http://localhost:8086"

client = influxdb_client.InfluxDBClient(
        url=url,
        token=token,
        org=org,
        timeout=60_000
    )

In [20]:
datafile = 'data/12-station-meteo-toulouse-montaudran.csv'
station = "12-montaudran"

def parse_row(row):
    return influxdb_client.Point("meteo") \
        .tag('station', station) \
        .field('temperature', float(row['temperature'])) \
        .field('pression', float(row['pression'])) \
        .field('pluie_intensite_max', float(row['pluie_intensite_max'])) \
        .field('force_moyenne_du_vecteur_vent', float(row['force_moyenne_du_vecteur_vent'])) \
        .time(row['heure_de_paris'])

data = rx \
    .from_iterable(DictReader(open(datafile, 'r'), delimiter=";")) \
    .pipe(ops.map(lambda row: parse_row(row)))

write_api = client.write_api(write_options=WriteOptions(batch_size=50_000, flush_interval=10_000))
write_api.write(bucket=bucket, org=org, record=data)
write_api.close()
print("Done !")

Done !


In [17]:
def delete_db():
    delete_api = client.delete_api()
    start = "1970-01-01T00:00:00Z"
    stop = "2022-02-01T00:00:00Z"
    delete_api.delete(start, stop, '_measurement="meteo"', bucket=bucket, org=org)
    print("Done !")
    
delete_db()

Done !
